In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
def _get_spark():
  spark = SparkSession.builder.appName("project_customer_360_campaign_test").getOrCreate()
  spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
  spark.conf.set("spark.sql.parquet.binaryAsString", "true")
  spark.conf.set("spark.sql.shuffle.partitions", 200)
  spark.conf.set("spark.sql.files.maxPartitionBytes", 1024*1024*256)
  return spark
spark = _get_spark()
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

In [3]:
# Below Command are executed only once to mount the blob as a mount volume on Databricks
# Mount Points:
# Blob Storage customer360-blob-data is available at mount point /mnt/customer360-blob-data
# Blob Storage customer360-blob-output is available at mount point /mnt/customer360-blob-output
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from pyspark.sql.functions import *

import sys
sys.version_info
spark.conf.set("spark.sql.parquet.binaryAsString","true")

In [4]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "DYNAMIC")
import datetime

dt = datetime.datetime(2019, 8, 1)
end = datetime.datetime(2020, 2, 14)
step = datetime.timedelta(days=1)

result = []
df = spark.read.parquet("dbfs:/mnt/customer360-blob-output/C360/PROFILE/l1_features/l1_customer_profile_union_daily_feature/event_partition_date=2020-02-14/")



In [5]:
while dt < end:
  partition_date = dt.strftime('%Y-%m-%d')
  result.append(partition_date)
  dt += step
  # only save whatever active before or at current date
  filtered_df = df.filter(col("register_date") <= to_date(lit(partition_date)))\
  .withColumn("event_partition_date", F.to_date(F.lit(partition_date)))\
  .withColumn("start_of_week", F.to_date(F.date_trunc('week', F.col("event_partition_date"))))\
  .withColumn("start_of_month", F.to_date(F.date_trunc('month', F.col("event_partition_date"))))
                          

  (filtered_df.write
   .partitionBy("event_partition_date")
   .mode("overwrite")
   .parquet("dbfs:/mnt/customer360-blob-output/C360/PROFILE/l1_features/l1_customer_profile_union_daily_feature/"))

  print(partition_date)